In [ ]:
import pandas as pd
import re

#Prompt for user input
file=input('What is the name of your .csv file?')
bank=input('What is the name of your Bank? Chose DanskeBank, Sydbank, N26 or DKB')

if bank == 'DanskeBank':
    df = pd.read_csv(file, delimiter=";", encoding='cp1252', usecols = ['Dato','Beløb','Tekst'])
    df.rename(columns={'Dato':'Date','Beløb':'Amount','Tekst':'Text'}, inplace=True)
    
elif bank == 'Sydbank':  
    df = pd.read_csv(file, delimiter=";", encoding='cp1252', usecols = ['Dato','Beløb','Tekst'])
    df.rename(columns={'Dato':'Date','Beløb':'Amount','Tekst':'Text'}, inplace=True)
      
    #Change german format to english one
    df['Amount'] = df['Amount'].replace({'\.':''}, regex = True)
    df['Amount'] = df['Amount'].replace({'\,':'.'}, regex = True)
    
    # Replace MCD and the numbers
    df['Text'] = df['Text'].replace({'^MCD':''}, regex = True)
    df['Text'] = df['Text'].replace({'\d*$':''}, regex = True)
    
elif bank == 'N26':  
    df = pd.read_csv(file, usecols = ['Datum','Verwendungszweck','Betrag (EUR)','Empfänger'])
    
    #merge empfänger and Verwendungszweck to 'Text'
    df['Verwendungszweck'].fillna(' ', inplace = True)
    df['Text'] = df['Empfänger'] +' '+ df['Verwendungszweck']
    
    df.rename(columns={'Datum':'Date','Betrag (EUR)':'Amount'}, inplace=True)
    df = df[['Text', 'Date', 'Amount']]               
    
    #Change Amount from Euro to DKK 
    df['Amount'] = df['Amount'].astype('float64', copy=False)
    df['Amount'] = df.Amount*7.5


elif bank =='DKB':
    df = pd.read_csv(file, delimiter=";", encoding='cp1252', usecols = ['Buchungstag','Betrag_EUR','Verwendungszweck'])
    df.rename(columns={'Buchungstag':'Date','Betrag_EUR':'Amount','Verwendungszweck':'Text'}, inplace=True)
    
    #replace kommas w/ dots -2.108,00
    df['Amount'] = df['Amount'].replace({'\.':''}, regex = True)
    df['Amount'] = df['Amount'].replace({'\,':'.'}, regex = True)
    
    #Replace AWV-MELDEPFLICHT BEACHTEN  HOTLINE BUNDESBANK: and (0800) 1234-111
    df['Text'] = df['Text'].replace({'(0800) 1234-111$':''}, regex = True)
    df['Text'] = df['Text'].replace({'AWV-MELDEPFLICHT BEACHTEN  HOTLINE BUNDESBANK:*':''}, regex = True) 
    
    #Change Amount from Euro to DKK 
    df['Amount'] = df['Amount'].astype('float64', copy=False)
    df['Amount'] = df.Amount*7.5

    #Delete dates in data
    for i in range(df.shape[0]):
        content=df['Text'][i]
        df['Text'][i]= re.sub('\s*(3[01]|[12][0-9]|0?[1-9])\.(1[012]|0?[1-9])\.((?:19|20)\d{2})\s(.*)', '',content)
        df['Text'][i]= re.sub('\s*(3[01]|[12][0-9]|0?[1-9])\.(1[012]|0?[1-9])\.\s(.*)', '',content)
        df['Text'][i]= re.sub('\s*(3[01]|[12][0-9]|0?[1-9])\.(1[012]|0?[1-9])\.\d{2}\s(.*)', '',content)

else:
    print('unknown Bank')

    
#Change Datatype of amount to float
df['Amount'] = df['Amount'].astype('float64', copy=False)
#Delete all positive transfers
df=df[df.Amount <0.0]
                     
#Delete weird brackets (((
df['Text'] = df['Text'].replace({'\)\)\)\)':''}, regex = True)
df['Text'] = df['Text'].replace({'\)\)\)':''}, regex = True)
                     
print(df.head())
                                       
                     
#Export the cleaned data
df.to_csv('Data_Cleaned.csv', index =False)

In [ ]:
## Remarks
 # How can we clean the data according to format (english or german)? Sydbank was in the same format, which is weird
#if number is in format *\d\.\d{2}/ then dont clen it, if its in *\d\.\d{2} 
 #Panda only reads 200 rows 
pattern=(\d*\.\d{2}$)
if pattern.match('Amount'):
    #replace kommas w/ dots 
    df['Amount'] = df['Amount'].replace({'\.':''}, regex = True)
    df['Amount'] = df['Amount'].replace({'\,':'.'}, regex = True)

#How do I turn around the date? 2018.11.02 -> 02.11.2018
    
    

In [ ]:
reader = pd.read_csv('tmp.sv', sep='|', chunksize=200)
for chunk in reader: 
    

In [6]:
#Prompt for user input
file='DanskeBank1.csv'
bank='DanskeBank'

if bank == 'DanskeBank':
    file = pd.read_csv(file, delimiter=";", encoding='cp1252', usecols = ['Dato','Beløb','Tekst'], chunksize=200)
    for df in file: 
        df.rename(columns={'Dato':'Date','Beløb':'Amount','Tekst':'Text'}, inplace=True)
        #Change Datatype of amount to float
        df['Amount'] = df['Amount'].astype('float64', copy=False)
            #Delete all positive transfers
        df=df[df.Amount <0.0]
                     
#Delete weird brackets (((
        df['Text'] = df['Text'].replace({'\)\)\)\)':''}, regex = True)
        df['Text'] = df['Text'].replace({'\)\)\)':''}, regex = True)
                     
        print(df.head())
                                       
                     
#Export the cleaned data
        df.to_csv('Data_Cleaned.csv', index =False, header=False)
    
else:
    print('unknown Bank')

#http://pandas-docs.github.io/pandas-docs-travis/user_guide/io.html#io-chunking

    


         Date                   Text  Amount
4  03.12.2018   Netto Fredriks Brygg   -19.5
5  04.12.2018   Spisestuerne ved Han   -15.0
6  05.12.2018       Braendte Mandler   -55.0
7  05.12.2018  Badi Is                 -50.0
8  05.12.2018        Lune Spanskroer   -40.0
           Date                   Text  Amount
200  10.04.2019  ITUniversitetet         -18.0
201  10.04.2019     Netto Axel Heidesg   -47.9
202  11.04.2019  ITUniversitetet         -18.5
203  11.04.2019  Netto Tranehavevej      -39.9
204  11.04.2019  Spisestuerne ved Han    -87.0
           Date                   Text  Amount
400  19.08.2019  Hallernes Smrrebrd      -68.0
401  19.08.2019  MOSSCPH ApS             -75.0
402  19.08.2019  Netto Nordhavn 7381     -91.8
403  19.08.2019  Phenomenon Aps         -250.0
404  20.08.2019  iZ *GAO Dumpling B     -140.0
